На страницах http://mediametrics.ru/data/archive/ размещены ежедневные архивы со "слепками" рейтинга новостей в течение дня.

На первом уровне идет разделение по тому, показатели за какой период находятся в слепках:

day - посещаемость за 24 часа;

hour - посещаемость за 1 час;

online - посещаемость за 10 минут.

Слепки с разными показателями делаются с разной частотой: с суточными показателями - раз в 5 минут, с часовыми - раз в минуту, с 10-минутными - раз в 10 секунд, поэтому самые большие архивы в каталоге online, в каждом архиве 8640 файлов-слепков.

Архивы создаются в течение нескольких минут после полуночи по московскому времени.

Внутри каталогов размещены ежедневные архивы по трем странам: России, Украине и Беларуси.

Например, рассмотрим архив http://mediametrics.ru/data/archive/day/ru-2014-04-01.zip
внутри него списки новостей с суточными показателями (day), по России (ru) за 1 апреля (2014-04-01).

В данном архиве 288 файлов (раз в 5 минут за день):
 day/ru-2014-04-01_00:00:00.tsv
 day/ru-2014-04-01_00:05:00.tsv
 day/ru-2014-04-01_00:10:00.tsv
и так далее. Каждый файл содержит список новостей на тот момент, который обозначен в имени файла (время московское), в списки записывается не более 500 новостей.

TSV - это текстовый формат данных, в котором колонки данных разделены символом табуляции.

В первой строчке находится обозначение колонок и служебные данные:

**URL  Title  Visitors  Delta  Shift  ID  68201  1396296000**

- URL - адрес страницы новости;
- Title - заголовок новости;
- Visitors - количество посетителей за выбранный период;
- Delta - разница в количестве посетителей с момента предыдущей генерации рейтинга (это частота отличается от частоты слепков за час и сутки, часовой и суточный рейтинг создаются раз в 10 и 30 секунд)
- Shift - условный показатель движения новости по позициям рейтинга вверх (меньше нуля) или вниз (больше нуля);
- ID - внутренний идентификатор новости, целое число (один идентификатор может быть присвоен разным новостям, появившихся с разницей более 3 месяцев);
- 68201 - общее количество новостей в списке, но в слепок попадают только первые 500;
- 1396296000 - время обновления рейтинга в виде так называемого unix timestamp (количество секунд с 1 января 1970 года)

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
import pandas as pd 
import matplotlib #collection of functions for scientific and publication-ready visualization
import numpy as np #foundational package for scientific computing
import scipy as sp #collection of functions for scientific computing and advance mathematics
import os, os.path
import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
import sklearn #collection of machine learning algorithms
import random
import time
import datetime
import urllib.request
import re
import warnings
warnings.filterwarnings('ignore')

import pickle
import zipfile
from tqdm import tqdm

from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib import request
from urllib.parse import quote

import jupyter_loader
import loader

importing Jupyter notebook from loader.ipynb
Wall time: 0 ns


In [2]:
# date_start = input('Enter a start date in YYYY-MM-DD format')
# date_end = input('Enter a end date in YYYY-MM-DD format')
date_start = '2018-10-03'
date_end = '2018-10-03'

date = loader.loader(date_start, date_end)
date.make_url()

In [3]:
import pandasmaker
#from imp import reload

#reload(pandasmaker)
pandas_file = pandasmaker.pandasmaker('./' + date.period)
pandas_file.make_pandas()

importing Jupyter notebook from pandasmaker.ipynb


100%|██████████| 2400/2400 [58:49<00:00,  3.35s/it]


In [19]:
names = pickle.load(open([name for name in os.listdir('./') if name.endswith('.pic')][0], 'rb'))
names.head()

,URL,Title,Visitors,Delta,Shift,ID,Date,Locate,Source_of_info
0,vesti.ru/doc.html?id=3066395,В России изменяются правила доставки пациентов...,15729,0,0,22557879,2018-10-03,ru,vesti.ru
1,novayagazeta.ru/articles/2018/09/29/77993-sud-...,Суд патриархата. Правосудие Тывы максимально л...,12178,0,0,49053795,2018-10-03,ru,novayagazeta.ru
2,www.360tv.ru/news/tekst/v-bagdade-ubili-samuju...,Убита самая красивая девушка Ирака. Причиной м...,10590,0,0,112769901,2018-10-03,ru,360tv.ru
3,www.mk.ru/moscow/2018/10/01/v-moskve-dvoe-poli...,В Москве двое полицейских расстреляли прохожег...,10401,1,0,113933922,2018-10-03,ru,mk.ru
4,www.kommersant.ru/doc/3753339,"«Умный, хороший, сообразительный»",10138,0,0,30517366,2018-10-03,ru,kommersant.ru


In [20]:
names.shape

(4800000, 9)

In [21]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800000 entries, 0 to 4799999
Data columns (total 9 columns):
URL               object
Title             object
Visitors          int64
Delta             int64
Shift             int64
ID                int64
Date              datetime64[ns]
Locate            object
Source_of_info    object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 329.6+ MB


In [22]:
import theme_loader

In [23]:
from imp import reload
reload(theme_loader)

theme = theme_loader.Theme_loader('сбербанк', names)
theme.num()

110

In [25]:
theme.data_articles()


 20%|██        | 22/110 [00:44<02:41,  1.83s/it]

HTTPError: HTTP Error 503: Service Unavailable

In [114]:
data = pickle.load(open([name for name in os.listdir('./') if name.endswith('.pcl')][0], 'rb'))
data

[['Фото: © РИА Новости/Кирилл Каллиников',
  'Фото: © РИА Новости/Кирилл Каллиников',
  '',
  'С недавних пор банки стали требовать от клиентов обоснований экономического смысла операций на небольшие суммы — в пределах 1 тысячи рублей. В случае отказа их карту или счёт могут заблокировать. Среди тех, кто решил подключиться к данному новшеству, как отмечается, оказался и Сбербанк.',
  '"Ничего мы вам не разблокируем"',
  '',
  'Клиенты уверяют: Сбер может заблокировать независимо от того, как долго вы пользуетесь его услугами. Так, один из московских бизнесменов написал на "Банки.ру" отзыв, что от него потребовали "пояснить экономический смысл операций". У него дебетовые карты Сбербанка для совершения оплат контрагентам по счетам и для оплат контрагентами вознаграждений за оказанные услуги. Кроме того, у него есть право на применение патентной системы налогообложения до 31.12.2018 года.',
  '',
  '— Я получил СМС следующего содержания: "Борис Владимирович, в соответствии с ФЗ №115-ФЗ от